{[Click here to read this notebook in Google Colab](https://colab.research.google.com/drive/1B-mCYmED_ePFCvYdYKwAAo49s87Nai4f)}

<head><link rel = "stylesheet" href = "https://drive.google.com/uc?id=1zYOH-_Mb9jOjRbQmghdhsmZ2g6xAwakk"></head>

<table class = "header"><tr>
    <th align = "left">EPAT Batch 45 | Final Project, July 2020</th>
    <th align = "right">Written by: Gaston Solari Loudet</th>
</tr></table>

### "Timer" class

After thorough research through different libraries related to time measurement and event handling, I've realized that there are not many implemented solutions for running tasks in the background at one constant frequency. We'll need this for different reasons. Some examples:
* <u>Data retrieval</u>: Execution of MT5 data request functions at a certain frequency.
<br>(Somewhat lower than tick data request, but higher than minutely)
* <u>Trade monitoring</u>: Position analysis according to market's actual situation.
<br>(Useful for tasks such as trailing stops, delay measurement or circuit breaking)
* <u>Connection monitoring</u>: Measuring of variables such as bandwidth or MT5 comm speed.

The closest thing to it, is the "[threading]("https://docs.python.org/3/library/threading.html")" library, commonly used as a basic parallel computing tool. Python is able to generate different operating "threads" that run independent from each other. Care must be taken when 2 threads interact requesting data from each other, as:
* Data may be incomplete as a thread may have been unable to complete its task.
* A busy thread can "become distracted" while sending data, increasing delays in its work.

The problem is that classes from "``threading``" package don't hold any method able to reset the execution once it's done: objects such as "``Thread``" or "``Timer``" only schedule tasks for future occasions to happen once.

#### Imports

We may re-use the aforementioned objects as "``threading``" package is commonly already installed in most Python environments. We could just create a "children" class that inherits attributes from the original class with the same name, and add the appropriate feats to suit our needs.

We shall also include "``time``" functions, as time measurements will be essential for measuring deviations from, say, "punctuality". It's better to leave "``datetime``" aside in this occasion for 2 reasons:

* "``timedelta``" formatting consumes much more memory, and could amplify unnecessary delays.
* Its precision is on the order of microseconds at max, and we need to measure in a CPU timeframe.

In [1]:
import threading, time

#### Constructor

The concept is largely based in two fundamental elements: a "``function``" to take place, and a time "``interval``" (step) between 2 consecutive executions. However, function may have its own arguments, isn't it? But we can't include keywords for them in the constructor: functions may vary, and arguments may not be the same.

Luckily, Python has quite a few feats to be taken advantage of:

* Functions with unspecified arguments are callback methods, able to be manipulated as objects (like "``lambda``").
<br>We can call function "``f``" without having to read a line containing "``f(x, y,...)``" or with eval-type functions.
<li>Retrieving optional input arguments by unpacking "``args``" and "``kwargs``" keywords with "``*``" (``list``) and "``**``" (``dict``) respectively.

We've already mentioned 4 things to be stored by a "``timer``" instance: "``interval``", "``function``", "``args``" and "``kwargs``". We need to give a "``threading.Timer``" primordial object to it too. Take notice that as it's intended to be manipulated from upcoming instance methods and not from its attribute directly, we shall label our "``_timer``" it with an underscore.

I will add 3 more variables to be explained later: "``is_running``" (``bool``ean), "``then``" (``float``) and "``delays``" (``list`` of floats).

In [2]:
class timer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self.interval, self.function, self.args, self.kwargs = interval, function, args, kwargs
        self._timer, self.is_running, self.then, self.delays = None, False, time.time(), []

#### Principle

The main problem with "``threading.Timer``" is that its timing function ceases to exist once its associated function has already run. We will need to re-create the object shall the function execute. But the function "``def``" doesn't have the timer creation on the inside. We'll then need to create some "``start``" method whose role would be essentially to execute the "``function``" as callback with its arguments (unpacked "``args``" and "``kwargs``") and then make the timer to start again. The "``_timer``" shall run **this** method as callback, not the "``function``" so as not to repeat the problem with "``threading``".

We shall control the timer activity with the "``is_running``" flag. When it is "``True``", timer shall be able to become reset after every loop. Otherwise, function wouldn't do anything. On the other hand, we shall add the required time measurement features:
* Before the upcoming timer reset, we shall take note of the actual "``time``" as "``then``".
* On the next cycle, the "``start``" and the actual "``function``" have been re-executed:
<br>...we shall record the difference between the "new" present and "``then``".
* The difference shall be stored in a list of "``delays``" with no more than the last 100 records.
<br>...this list can be useful for later comparison.</li></ol></body>

In [3]:
class timer(timer):
    def __init__(self, interval, function, *args, **kwargs):
        super().__init__(interval, function, *args, **kwargs)
    def start(self):
        if self.is_running:
            self.function(*self.args, **self.kwargs)
            self.delays.append(time.time() - self.then)
            if (len(self.delays) > 100): self.delays.pop(0)
            self.then = time.time()
            self._timer = threading.Timer(self.interval, self.start)
            self._timer.start() # Switch "threading.Timer" instance on.

Note that a "``threading.Timer``" needs its own "``start``" command to begin counting time until run. This utility must end with such call. On the other hand, we need a different method for the opposite action: it's not enough just to unset "``is_running``" from the outside because the original timer object needs a "``cancel``" command to stop as well. Our next "``stop``" method will take care of this.

In [4]:
class timer(timer):
    def __init__(self, interval, function, *args, **kwargs):
        super().__init__(interval, function, *args, **kwargs)
    def stop(self):
        self._timer.cancel()
        self.is_running = False

As an additional feat to ease the testing, we could add the following dunder to be able to run and stop the timer by only typing its name in the command line. It shall return the mean of all the core function's execution times.

In [5]:
class timer(timer):
    def __init__(self, interval, function, *args, **kwargs):
        super().__init__(interval, function, *args, **kwargs)
    def __repr__(self):
        if self.is_running: self.stop()
        else: self.is_running = True; self.start()
        return str(sum(self.delays)/len(self.delays))

#### Unit tests

Our timer class construction is now complete. Now we shall write a few trial functions for correct manipulation and execution time measurement. Let's choose 3 different examples with varying execution times to see how much does the timer delay in its reset because of running the particular "``function``" itself:
    
1. "Easy" (about $30\;us$): function just "``return``s" and does nothing else.
2. "Medium" (about $4500\;us$): "``print``s" the actual time in "``time``" library format.
3. "Hard" (about $90000\;us$): given a large initial "$p$", it will repeatedly calculate "$p^{0.999}$".
Such times will **largely** depend on factors such as computer's CPU/OS/RAM, if other tasks are being done, etc. Also times will likely be larger with timers being executed here as "``.ipynb``" files are more complex than "``.py``".

In [6]:
def timer_test_easy():
    def f(): return
    return timer(interval = 1/10, function = f)
def timer_test_medium():
    def f(): print(time.time())
    return timer(interval = 1/10, function = f)
def timer_test_hard():
    def f():
        p = 1e100
        for n in range(100000): p = p**0.999
    return timer(interval = 1/10, function = f)

#### Conclusions

The class has successfully accomplished its role, and will be applied in future requirements. There are a few considerations to bear in mind:

* After some experimentation, I found the delay to be <b>independent</b> on the selected "``interval``" itself, in all cases above. Luckily, this grants us the freedom to adjust the timer frequency while only considering the provided "``function``" runtime.
* Let "$\Delta t_{del}$" be the average delay, and "$\Delta t_{int}$" the selected "``interval``". If "$\Delta t_{int} / \Delta t_{del} = n$", that means that the system will erroneously skip/omit one execution after around "$n$" ones. Logically, "$n$" must be as large as possible.

In [7]:
x = timer_test_medium()